In [1]:
import torch
import sys
sys.path.insert(0, '../src')
import wandb
import pickle as pkl
import numpy as np
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader

from data_generator import SpatialDataset, train_val_test_split
from models import LinearSCI, NonlinearSCI
from trainers import Trainer

# Experimental tracking
wandb.login()

np.random.seed(2023)
torch.manual_seed(2023)
torch.cuda.manual_seed(2023)
torch.cuda.manual_seed_all(2023)
torch.backends.cudnn.deterministic = True

wandb: Currently logged in as: ziyang-jiang (carlsonlab). Use `wandb login --relogin` to force relogin


# Load data

In [2]:
with open('../data/synthetic_data.pkl', 'rb') as fp:
    data = pkl.load(fp)
neighborhood_size = data['neighborhood_size']
T = np.concatenate(
    [data['T_bar'][:,:neighborhood_size], 
     data['T'], 
     data['T_bar'][:,neighborhood_size:]], axis=1)
X, Y, s = data['X'], data['Y'], data['s']
de_0, de_1, ie_0, ie_1, te = data['de_0'], data['de_1'], data['ie_0'], data['ie_1'], data['te']

train_dataset, val_dataset, test_dataset = train_val_test_split(
    t=[T], x=X, s=s, y=Y, train_size=0.6, val_size=0.2, test_size=0.2, 
    shuffle=True, random_state=2020
)
train_loader = DataLoader(train_dataset, batch_size=50, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=50, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=50, shuffle=False)
dataloaders = {'train': train_loader, 'val': val_loader, 'test': test_loader}

# Method 1: Linear model without U

In [ ]:
num_iterations = 5
de_error_0, de_error_1, ie_error_0, ie_error_1, te_error = [], [], [], [], []

for _ in tqdm(range(num_iterations), position=0, leave=True):
    model = LinearSCI(
        num_interventions=1, 
        window_size=neighborhood_size*2+1, 
        confounder_dim=X.shape[1]
    )
    
    # Training
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optim = "sgd"
    optim_params = {
        'lr': 1e-3, 
        'momentum': 0.99
    }
    epochs, patience = 1000, 50
    trainer = Trainer(
        model=model, 
        data_generators=dataloaders, 
        optim=optim, 
        optim_params=optim_params, 
        device=device,
        epochs=epochs,
        patience=patience
    )
    trainer.train()

    # Evaluation
    y_pred_00 = trainer.predict(neighborhood_size*2+1, direct=False, indirect=False)
    y_pred_01 = trainer.predict(neighborhood_size*2+1, direct=False, indirect=True)
    y_pred_10 = trainer.predict(neighborhood_size*2+1, direct=True, indirect=False)
    y_pred_11 = trainer.predict(neighborhood_size*2+1, direct=True, indirect=True)
    de_0_pred, de_1_pred = np.mean(y_pred_10 - y_pred_00), np.mean(y_pred_11 - y_pred_01)
    ie_0_pred, ie_1_pred = np.mean(y_pred_01 - y_pred_00), np.mean(y_pred_11 - y_pred_10)
    te_pred = np.mean(y_pred_11 - y_pred_00)
    de_error_0.append(np.abs(de_0_pred - de_0))
    de_error_1.append(np.abs(de_1_pred - de_1))
    ie_error_0.append(np.abs(ie_0_pred - ie_0))
    ie_error_1.append(np.abs(ie_1_pred - ie_1))
    te_error.append(np.abs(te_pred - te))

In [4]:
print("Error on prediction of average local and interference effects:")
print("--------------------------------------------------------------")
print(f"Average direct effect (T_bar = 0) = {np.mean(de_error_0):.5f} +/- {np.std(de_error_0):.5f}")
print(f"Average direct effect (T_bar = 1) = {np.mean(de_error_1):.5f} +/- {np.std(de_error_1):.5f}")
print(f"Average indirect effect (T = 0) = {np.mean(ie_error_0):.5f} +/- {np.std(ie_error_0):.5f}")
print(f"Average indirect effect (T = 1) = {np.mean(ie_error_1):.5f} +/- {np.std(ie_error_1):.5f}")
print(f"Average total effect = {np.mean(te_error):.5f} +/- {np.std(te_error):.5f}")

Error on prediction of average local and interference effects:
--------------------------------------------------------------
Average direct effect (T_bar = 0) = 1.01948 +/- 1.03605
Average direct effect (T_bar = 1) = 1.01948 +/- 1.03605
Average indirect effect (T = 0) = 0.79898 +/- 1.03110
Average indirect effect (T = 1) = 0.79898 +/- 1.03110
Average total effect = 0.22774 +/- 0.00336


# Model 2: Linear model with U

In [ ]:
num_iterations = 5
de_error_0, de_error_1, ie_error_0, ie_error_1, te_error = [], [], [], [], []

for _ in tqdm(range(num_iterations), position=0, leave=True):
    model = LinearSCI(
        num_interventions=1, 
        window_size=neighborhood_size*2+1, 
        confounder_dim=X.shape[1], 
        unobserved_confounder=True, 
        kernel_param_vals=[1.,0.5,0.1]
    )
    
    # Training
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optim = "adam"
    optim_params = {
        'lr': 1e-3, 
        # 'momentum': 0.99
    }
    epochs, patience = 1000, 50
    trainer = Trainer(
        model=model, 
        data_generators=dataloaders, 
        optim=optim, 
        optim_params=optim_params, 
        device=device,
        epochs=epochs,
        patience=patience
    )
    trainer.train()

    # Evaluation
    y_pred_00 = trainer.predict(neighborhood_size*2+1, direct=False, indirect=False)
    y_pred_01 = trainer.predict(neighborhood_size*2+1, direct=False, indirect=True)
    y_pred_10 = trainer.predict(neighborhood_size*2+1, direct=True, indirect=False)
    y_pred_11 = trainer.predict(neighborhood_size*2+1, direct=True, indirect=True)
    de_0_pred, de_1_pred = np.mean(y_pred_10 - y_pred_00), np.mean(y_pred_11 - y_pred_01)
    ie_0_pred, ie_1_pred = np.mean(y_pred_01 - y_pred_00), np.mean(y_pred_11 - y_pred_10)
    te_pred = np.mean(y_pred_11 - y_pred_00)
    de_error_0.append(np.abs(de_0_pred - de_0))
    de_error_1.append(np.abs(de_1_pred - de_1))
    ie_error_0.append(np.abs(ie_0_pred - ie_0))
    ie_error_1.append(np.abs(ie_1_pred - ie_1))
    te_error.append(np.abs(te_pred - te))

In [7]:
print("Error on prediction of average local and interference effects:")
print("--------------------------------------------------------------")
print(f"Average direct effect (T_bar = 0) = {np.mean(de_error_0):.5f} +/- {np.std(de_error_0):.5f}")
print(f"Average direct effect (T_bar = 1) = {np.mean(de_error_1):.5f} +/- {np.std(de_error_1):.5f}")
print(f"Average indirect effect (T = 0) = {np.mean(ie_error_0):.5f} +/- {np.std(ie_error_0):.5f}")
print(f"Average indirect effect (T = 1) = {np.mean(ie_error_1):.5f} +/- {np.std(ie_error_1):.5f}")
print(f"Average total effect = {np.mean(te_error):.5f} +/- {np.std(te_error):.5f}")

Error on prediction of average local and interference effects:
--------------------------------------------------------------
Average direct effect (T_bar = 0) = 0.62597 +/- 0.53641
Average direct effect (T_bar = 1) = 0.62597 +/- 0.53641
Average indirect effect (T = 0) = 0.25645 +/- 0.18804
Average indirect effect (T = 1) = 0.25645 +/- 0.18804
Average total effect = 0.46593 +/- 0.30781


# Model 3: Nonlinear model without U (f: MLP, g:MLP)

In [ ]:
num_iterations = 5
de_error_0, de_error_1, ie_error_0, ie_error_1, te_error = [], [], [], [], []

for _ in tqdm(range(num_iterations), position=0, leave=True):
    model = NonlinearSCI(
        num_interventions=1, 
        window_size=neighborhood_size*2+1, 
        confounder_dim=X.shape[1], 
        f_network_type="mlp", 
        f_hidden_dims=[64,32], 
        g_network_type="mlp", 
        g_hidden_dims=[64,32]
    )
    
    # Training
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optim = "sgd"
    optim_params = {
        'lr': 1e-3, 
        'momentum': 0.99
    }
    epochs, patience = 1000, 50
    trainer = Trainer(
        model=model, 
        data_generators=dataloaders, 
        optim=optim, 
        optim_params=optim_params, 
        device=device,
        epochs=epochs,
        patience=patience
    )
    trainer.train()

    # Evaluation
    y_pred_00 = trainer.predict(neighborhood_size*2+1, direct=False, indirect=False)
    y_pred_01 = trainer.predict(neighborhood_size*2+1, direct=False, indirect=True)
    y_pred_10 = trainer.predict(neighborhood_size*2+1, direct=True, indirect=False)
    y_pred_11 = trainer.predict(neighborhood_size*2+1, direct=True, indirect=True)
    de_0_pred, de_1_pred = np.mean(y_pred_10 - y_pred_00), np.mean(y_pred_11 - y_pred_01)
    ie_0_pred, ie_1_pred = np.mean(y_pred_01 - y_pred_00), np.mean(y_pred_11 - y_pred_10)
    te_pred = np.mean(y_pred_11 - y_pred_00)
    de_error_0.append(np.abs(de_0_pred - de_0))
    de_error_1.append(np.abs(de_1_pred - de_1))
    ie_error_0.append(np.abs(ie_0_pred - ie_0))
    ie_error_1.append(np.abs(ie_1_pred - ie_1))
    te_error.append(np.abs(te_pred - te))

In [9]:
print("Error on prediction of average local and interference effects:")
print("--------------------------------------------------------------")
print(f"Average direct effect (T_bar = 0) = {np.mean(de_error_0):.5f} +/- {np.std(de_error_0):.5f}")
print(f"Average direct effect (T_bar = 1) = {np.mean(de_error_1):.5f} +/- {np.std(de_error_1):.5f}")
print(f"Average indirect effect (T = 0) = {np.mean(ie_error_0):.5f} +/- {np.std(ie_error_0):.5f}")
print(f"Average indirect effect (T = 1) = {np.mean(ie_error_1):.5f} +/- {np.std(ie_error_1):.5f}")
print(f"Average total effect = {np.mean(te_error):.5f} +/- {np.std(te_error):.5f}")

Error on prediction of average local and interference effects:
--------------------------------------------------------------
Average direct effect (T_bar = 0) = 0.24465 +/- 0.17476
Average direct effect (T_bar = 1) = 0.24465 +/- 0.17476
Average indirect effect (T = 0) = 0.01251 +/- 0.01698
Average indirect effect (T = 1) = 0.01251 +/- 0.01698
Average total effect = 0.25445 +/- 0.18840


# Model 4: Nonlinear model with U (f: MLP, g: MLP)

In [ ]:
num_iterations = 5
de_error_0, de_error_1, ie_error_0, ie_error_1, te_error = [], [], [], [], []

for _ in tqdm(range(num_iterations), position=0, leave=True):
    model = NonlinearSCI(
        num_interventions=1, 
        window_size=neighborhood_size*2+1, 
        confounder_dim=X.shape[1], 
        f_network_type="mlp", 
        f_hidden_dims=[64,32], 
        g_network_type="mlp", 
        g_hidden_dims=[64,32], 
        unobserved_confounder=True, 
        kernel_param_vals=[1.,0.5,0.1]
    )
    
    # Training
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optim = "adam"
    optim_params = {
        'lr': 1e-3, 
        # 'momentum': 0.99
    }
    epochs, patience = 1000, 50
    trainer = Trainer(
        model=model, 
        data_generators=dataloaders, 
        optim=optim, 
        optim_params=optim_params, 
        device=device,
        epochs=epochs,
        patience=patience
    )
    trainer.train()

    # Evaluation
    y_pred_00 = trainer.predict(neighborhood_size*2+1, direct=False, indirect=False)
    y_pred_01 = trainer.predict(neighborhood_size*2+1, direct=False, indirect=True)
    y_pred_10 = trainer.predict(neighborhood_size*2+1, direct=True, indirect=False)
    y_pred_11 = trainer.predict(neighborhood_size*2+1, direct=True, indirect=True)
    de_0_pred, de_1_pred = np.mean(y_pred_10 - y_pred_00), np.mean(y_pred_11 - y_pred_01)
    ie_0_pred, ie_1_pred = np.mean(y_pred_01 - y_pred_00), np.mean(y_pred_11 - y_pred_10)
    te_pred = np.mean(y_pred_11 - y_pred_00)
    de_error_0.append(np.abs(de_0_pred - de_0))
    de_error_1.append(np.abs(de_1_pred - de_1))
    ie_error_0.append(np.abs(ie_0_pred - ie_0))
    ie_error_1.append(np.abs(ie_1_pred - ie_1))
    te_error.append(np.abs(te_pred - te))

In [11]:
print("Error on prediction of average local and interference effects:")
print("--------------------------------------------------------------")
print(f"Average direct effect (T_bar = 0) = {np.mean(de_error_0):.5f} +/- {np.std(de_error_0):.5f}")
print(f"Average direct effect (T_bar = 1) = {np.mean(de_error_1):.5f} +/- {np.std(de_error_1):.5f}")
print(f"Average indirect effect (T = 0) = {np.mean(ie_error_0):.5f} +/- {np.std(ie_error_0):.5f}")
print(f"Average indirect effect (T = 1) = {np.mean(ie_error_1):.5f} +/- {np.std(ie_error_1):.5f}")
print(f"Average total effect = {np.mean(te_error):.5f} +/- {np.std(te_error):.5f}")

Error on prediction of average local and interference effects:
--------------------------------------------------------------
Average direct effect (T_bar = 0) = 0.18627 +/- 0.17602
Average direct effect (T_bar = 1) = 0.18627 +/- 0.17602
Average indirect effect (T = 0) = 0.25104 +/- 0.08557
Average indirect effect (T = 1) = 0.25104 +/- 0.08557
Average total effect = 0.24088 +/- 0.13501


# Model 5: GCN model without U (f: GCN, g: MLP)

In [12]:
train_dataset, val_dataset, test_dataset = train_val_test_split(
    t=[T], x=X, s=s, y=Y, train_size=0.6, val_size=0.2, test_size=0.2, 
    shuffle=True, random_state=2020, graph_input=True
)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
dataloaders = {'train': train_loader, 'val': val_loader, 'test': test_loader}

100%|█████████████████████████████████████| 100/100 [00:00<00:00, 346350.45it/s]


In [ ]:
num_iterations = 5
de_error_0, de_error_1, ie_error_0, ie_error_1, te_error = [], [], [], [], []

for _ in tqdm(range(num_iterations), position=0, leave=True):
    model = NonlinearSCI(
        num_interventions=1, 
        window_size=neighborhood_size*2+1, 
        confounder_dim=X.shape[1], 
        f_network_type="gcn", 
        g_network_type="mlp", 
        g_hidden_dims=[64,32]
    )
    
    # Training
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optim = "sgd"
    optim_params = {
        'lr': 1e-3, 
        'momentum': 0.99
    }
    epochs, patience = 1000, 50
    trainer = Trainer(
        model=model, 
        data_generators=dataloaders, 
        optim=optim, 
        optim_params=optim_params, 
        device=device,
        epochs=epochs,
        patience=patience
    )
    trainer.train()

    # Evaluation
    y_pred_00 = trainer.predict(neighborhood_size*2+1, direct=False, indirect=False)
    y_pred_01 = trainer.predict(neighborhood_size*2+1, direct=False, indirect=True)
    y_pred_10 = trainer.predict(neighborhood_size*2+1, direct=True, indirect=False)
    y_pred_11 = trainer.predict(neighborhood_size*2+1, direct=True, indirect=True)
    de_0_pred, de_1_pred = np.mean(y_pred_10 - y_pred_00), np.mean(y_pred_11 - y_pred_01)
    ie_0_pred, ie_1_pred = np.mean(y_pred_01 - y_pred_00), np.mean(y_pred_11 - y_pred_10)
    te_pred = np.mean(y_pred_11 - y_pred_00)
    de_error_0.append(np.abs(de_0_pred - de_0))
    de_error_1.append(np.abs(de_1_pred - de_1))
    ie_error_0.append(np.abs(ie_0_pred - ie_0))
    ie_error_1.append(np.abs(ie_1_pred - ie_1))
    te_error.append(np.abs(te_pred - te))

In [14]:
print("Error on prediction of average local and interference effects:")
print("--------------------------------------------------------------")
print(f"Average direct effect (T_bar = 0) = {np.mean(de_error_0):.5f} +/- {np.std(de_error_0):.5f}")
print(f"Average direct effect (T_bar = 1) = {np.mean(de_error_1):.5f} +/- {np.std(de_error_1):.5f}")
print(f"Average indirect effect (T = 0) = {np.mean(ie_error_0):.5f} +/- {np.std(ie_error_0):.5f}")
print(f"Average indirect effect (T = 1) = {np.mean(ie_error_1):.5f} +/- {np.std(ie_error_1):.5f}")
print(f"Average total effect = {np.mean(te_error):.5f} +/- {np.std(te_error):.5f}")

Error on prediction of average local and interference effects:
--------------------------------------------------------------
Average direct effect (T_bar = 0) = 1.02778 +/- 0.13583
Average direct effect (T_bar = 1) = 1.03302 +/- 0.08752
Average indirect effect (T = 0) = 0.04588 +/- 0.03320
Average indirect effect (T = 1) = 0.05374 +/- 0.03742
Average total effect = 1.07890 +/- 0.10665


# Model 6: GCN model with U (f: GCN, g: MLP)

In [ ]:
num_iterations = 5
de_error_0, de_error_1, ie_error_0, ie_error_1, te_error = [], [], [], [], []

for _ in tqdm(range(num_iterations), position=0, leave=True):
    model = NonlinearSCI(
        num_interventions=1, 
        window_size=neighborhood_size*2+1, 
        confounder_dim=X.shape[1], 
        f_network_type="gcn", 
        g_network_type="mlp", 
        g_hidden_dims=[64,32], 
        unobserved_confounder=True, 
        kernel_param_vals=[1.,0.5,0.1]
    )
    
    # Training
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optim = "adam"
    optim_params = {
        'lr': 1e-3, 
        # 'momentum': 0.99
    }
    epochs, patience = 1000, 50
    trainer = Trainer(
        model=model, 
        data_generators=dataloaders, 
        optim=optim, 
        optim_params=optim_params, 
        device=device,
        epochs=epochs,
        patience=patience
    )
    trainer.train()

    # Evaluation
    y_pred_00 = trainer.predict(neighborhood_size*2+1, direct=False, indirect=False)
    y_pred_01 = trainer.predict(neighborhood_size*2+1, direct=False, indirect=True)
    y_pred_10 = trainer.predict(neighborhood_size*2+1, direct=True, indirect=False)
    y_pred_11 = trainer.predict(neighborhood_size*2+1, direct=True, indirect=True)
    de_0_pred, de_1_pred = np.mean(y_pred_10 - y_pred_00), np.mean(y_pred_11 - y_pred_01)
    ie_0_pred, ie_1_pred = np.mean(y_pred_01 - y_pred_00), np.mean(y_pred_11 - y_pred_10)
    te_pred = np.mean(y_pred_11 - y_pred_00)
    de_error_0.append(np.abs(de_0_pred - de_0))
    de_error_1.append(np.abs(de_1_pred - de_1))
    ie_error_0.append(np.abs(ie_0_pred - ie_0))
    ie_error_1.append(np.abs(ie_1_pred - ie_1))
    te_error.append(np.abs(te_pred - te))

In [15]:
print("Error on prediction of average local and interference effects:")
print("--------------------------------------------------------------")
print(f"Average direct effect (T_bar = 0) = {np.mean(de_error_0):.5f} +/- {np.std(de_error_0):.5f}")
print(f"Average direct effect (T_bar = 1) = {np.mean(de_error_1):.5f} +/- {np.std(de_error_1):.5f}")
print(f"Average indirect effect (T = 0) = {np.mean(ie_error_0):.5f} +/- {np.std(ie_error_0):.5f}")
print(f"Average indirect effect (T = 1) = {np.mean(ie_error_1):.5f} +/- {np.std(ie_error_1):.5f}")
print(f"Average total effect = {np.mean(te_error):.5f} +/- {np.std(te_error):.5f}")

Error on prediction of average local and interference effects:
--------------------------------------------------------------
Average direct effect (T_bar = 0) = 0.80446 +/- 0.66531
Average direct effect (T_bar = 1) = 1.03512 +/- 0.94692
Average indirect effect (T = 0) = 0.14094 +/- 0.07259
Average indirect effect (T = 1) = 0.34441 +/- 0.17961
Average total effect = 1.01789 +/- 0.88741
